<a href="https://colab.research.google.com/github/RamaKrisnakommineni/INFO7374/blob/master/Assignment2/Part%202/A2Exp6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Experiment 6: Fine tuning
###Apply fine tuning for your team’s network to the CIFAR 10 dataset.

In [0]:
import numpy as np
from keras.datasets import cifar10
from keras.utils import to_categorical
(X_train,y_train),(X_test,y_test) = cifar10.load_data()
X_train = X_train/255
X_test = X_test/255

num_classes = len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

Using TensorFlow backend.


170500096/170498071 [==============================] - 35s 0us/step


###Importing DenseNet169 from keras

In [0]:
from keras.applications import DenseNet169
##Load model
base_model = DenseNet169(include_top=False,input_shape=(32,32,3))

Instructions for updating:
Colocations handled automatically by placer.
51879936/51877672 [==============================] - 2s 0us/step


In [0]:
#Summary of the base DenseNet169
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 16, 16, 64)   9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

###Experiments were performed by unfreezing last, last two, last three convolutional layers.

In [0]:
for layer in base_model.layers[:-11]: ##[:-4],[:-7]
  layer.trainable = False
for layer in base_model.layers:  ## To see the status of layers
  print(layer,layer.trainable)


<keras.engine.input_layer.InputLayer object at 0x7f2726a41e80> False
<keras.layers.convolutional.ZeroPadding2D object at 0x7f2726a662b0> False
<keras.layers.convolutional.Conv2D object at 0x7f2726a66160> False
<keras.layers.normalization.BatchNormalization object at 0x7f2726a66278> False
<keras.layers.core.Activation object at 0x7f2726a66358> False
<keras.layers.convolutional.ZeroPadding2D object at 0x7f26e2f4b080> False
<keras.layers.pooling.MaxPooling2D object at 0x7f26daee3b70> False
<keras.layers.normalization.BatchNormalization object at 0x7f26e3a24a90> False
<keras.layers.core.Activation object at 0x7f26dae79d68> False
<keras.layers.convolutional.Conv2D object at 0x7f26dae52cf8> False
<keras.layers.normalization.BatchNormalization object at 0x7f26dae32940> False
<keras.layers.core.Activation object at 0x7f26dadf2470> False
<keras.layers.convolutional.Conv2D object at 0x7f26dad26d68> False
<keras.layers.merge.Concatenate object at 0x7f26dace0a20> False
<keras.layers.normalization.

In [0]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()

model.add(base_model)

##Add new layer

model.add(layers.Flatten())
model.add(layers.Dense(1024, activation = 'relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(10,activation='softmax'))

model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet169 (Model)          (None, 1, 1, 1664)        12642880  
_________________________________________________________________
flatten_1 (Flatten)          (None, 1664)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1704960   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                10250     
Total params: 14,358,090
Trainable params: 2,004,682
Non-trainable params: 12,353,408
_________________________________________________________________


In [0]:
model.compile(loss='categorical_crossentropy',
             optimizer = optimizers.Adam(lr=1e-5), metrics=['acc'])

In [0]:
history = model.fit(X_train,y_train,
                   batch_size = 500,
                   epochs=30,
                   verbose=1,
                   validation_data=(X_test,y_test),shuffle=True)

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 41s 812us/step - loss: 2.4342 - acc: 0.1242 - val_loss: 3.1919 - val_acc: 0.1484
Epoch 2/30
50000/50000 [==============================] - 23s 465us/step - loss: 2.2720 - acc: 0.1787 - val_loss: 2.8668 - val_acc: 0.2023
Epoch 3/30
50000/50000 [==============================] - 23s 465us/step - loss: 2.1481 - acc: 0.2298 - val_loss: 2.6543 - val_acc: 0.2487
Epoch 4/30
50000/50000 [==============================] - 23s 464us/step - loss: 2.0295 - acc: 0.2897 - val_loss: 2.5485 - val_acc: 0.2802
Epoch 5/30
50000/50000 [==============================] - 23s 465us/step - loss: 1.9213 - acc: 0.3343 - val_loss: 2.5279 - val_acc: 0.3055
Epoch 6/30
50000/50000 [==============================] - 23s 464us/step - loss: 1.8173 - acc: 0.3790 - val_loss: 2.5150 - val_acc: 0.3272
Epoch 7/30
50000/50000 [==============================] - 23s 465us/